# CLARITY: SemEval 2026 — A100 Training

| Config | Model | Precision | VRAM | Expected F1 |
|--------|-------|-----------|------|-------------|
| `deberta_v3_base.yaml` | DeBERTa-v3-base | auto(bf16) | ~6GB | ~0.48–0.56 |
| `deberta_v3_large.yaml` | DeBERTa-v3-large | bf16 | ~12-14GB | ~0.56–0.65 |

## 1. Setup

In [24]:
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    props = torch.cuda.get_device_properties(0)
    vram = getattr(props, 'total_memory', getattr(props, 'total_mem', 0))
    print(f"VRAM: {vram / 1e9:.1f} GB")
    print(f"bf16: {torch.cuda.is_bf16_supported()}")

PyTorch: 2.10.0+cu128
CUDA: True
GPU: NVIDIA A100-SXM4-80GB
VRAM: 85.1 GB
bf16: True


In [25]:
import os, sys

try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    REPO_URL = "https://github.com/wilsebbis/semeval.git"
    if not os.path.exists('/content/semeval'):
        !git clone {REPO_URL} /content/semeval
    else:
        !cd /content/semeval && git pull
    os.chdir('/content/semeval')
    %pip install -e ".[dev]"
    os.environ['HF_HOME'] = '/content/hf_cache'
else:
    if os.path.basename(os.getcwd()) == 'notebooks':
        os.chdir(os.path.join(os.getcwd(), '..'))

src_path = os.path.join(os.getcwd(), 'src')
if src_path not in sys.path:
    sys.path.insert(0, src_path)

print(f"Working dir: {os.getcwd()}")
from clarity.labels import EVASION_LABELS, validate_labels
validate_labels()
print(f"✓ {len(EVASION_LABELS)} evasion labels")

remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 8 (delta 4), reused 8 (delta 4), pack-reused 0 (from 0)
Unpacking objects: 100% (8/8), 9.44 KiB | 2.36 MiB/s, done.
From https://github.com/wilsebbis/semeval
   5171636..4341fb0  main       -> origin/main
Updating 5171636..4341fb0
Fast-forward
 notebooks/train_colab.ipynb | 528 ++++++++++++++++++++++++++++++++++++++++++--
 src/clarity/models.py       |  21 +-
 src/clarity/train.py        |   7 +-
 3 files changed, 529 insertions(+), 27 deletions(-)
Obtaining file:///content/semeval
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for clarity-semeval2026 (pyproject.toml) ... done
  Created wheel for clarity-semeval2026: filen

## 2. Data

In [26]:
from pathlib import Path
if not Path('data/train.csv').exists():
    !git clone https://huggingface.co/datasets/ailsntua/QEvasion 2>/dev/null || true
    !bash scripts/prepare_data.sh
else:
    print("Data ready.")

Data ready.


In [27]:
import pandas as pd
train = pd.read_csv('data/train.csv')
dev = pd.read_csv('data/dev.csv')
print(f"Train: {len(train)} | Dev: {len(dev)}")
print(train['evasion_label'].value_counts().to_string())

Train: 2930 | Dev: 518
evasion_label
Explicit               894
Dodging                600
Implicit               415
General                328
Deflection             324
Declining to answer    123
Claims ignorance       101
Clarification           78
Partial/half-answer     67


## 3. Train

In [28]:
# ── Pick config ──
# CONFIG = "configs/deberta_v3_base.yaml"           # T4/Mac
CONFIG = "configs/deberta_v3_large.yaml"             # A100
# CONFIG = "configs/deberta_v3_large_a100.yaml"      # A100 alt

import yaml
with open(CONFIG) as f:
    cfg = yaml.safe_load(f)
print(f"Config: {CONFIG}")
for k, v in sorted(cfg.items()):
    print(f"  {k}: {v}")

Config: configs/deberta_v3_large.yaml
  alpha: 0.7
  batch_size: 8
  consistency_beta: 0.1
  dropout: 0.1
  epochs: 5
  fp16: False
  grad_accum: 2
  label_smoothing: 0.05
  lr: 1e-05
  max_length: 384
  model_name: microsoft/deberta-v3-large
  output_dir: checkpoints/deberta_v3_large
  patience: 3
  precision: bf16
  seed: 42
  task: evasion
  use_class_weights: False
  use_focal_loss: False
  warmup_ratio: 0.06
  weight_decay: 0.01


In [29]:
!python -m clarity.train \
    --config {CONFIG} \
    --data data/train.csv \
    --dev data/dev.csv \
    --task evasion \
    --debug_text

[2026-02-20 06:38:38] INFO clarity: Configuration: {'config': 'configs/deberta_v3_large.yaml', 'data': 'data/train.csv', 'dev': 'data/dev.csv', 'task': 'evasion', 'model_name': 'microsoft/deberta-v3-large', 'max_length': 384, 'batch_size': 8, 'grad_accum': 2, 'lr': 1e-05, 'weight_decay': 0.01, 'warmup_ratio': 0.06, 'epochs': 5, 'patience': 3, 'dropout': 0.1, 'seed': 42, 'output_dir': 'checkpoints/deberta_v3_large', 'use_focal_loss': False, 'focal_gamma': 2.0, 'use_class_weights': False, 'alpha': 0.7, 'consistency_beta': 0.1, 'num_workers': 0, 'fp16': False, 'precision': 'bf16', 'device': None, 'label_smoothing': 0.05, 'debug_text': True}
[2026-02-20 06:38:38] INFO clarity: Using device: cuda
[2026-02-20 06:38:38] INFO clarity: CUDA: TF32 enabled for matmul and cuDNN
[2026-02-20 06:38:38] INFO clarity: CUDA: Flash/mem-efficient/math SDP all enabled
[2026-02-20 06:38:38] INFO clarity: GPU: NVIDIA A100-SXM4-80GB (85.1 GB)
[2026-02-20 06:38:38] INFO clarity: Precision: bf16 | Attention: sd

## 4. Evaluate

In [30]:
import json
output_dir = cfg['output_dir']
mp = Path(output_dir) / 'metrics.json'
if mp.exists():
    m = json.load(open(mp))
    for h in m.get('history', []):
        print(f"  Epoch {h.get('epoch','?')}: F1={h.get('evasion_macro_f1',0):.4f}, loss={h.get('val_loss',0):.4f}")
    print(f"Best F1: {m.get('best_f1', 0):.4f}")
else:
    print('No metrics found.')

No metrics found.


## 5. Predict

In [31]:
!python -m clarity.predict \
    --ckpt {output_dir}/best_model.pt \
    --data data/dev.csv \
    --out submissions/predictions.csv \
    --evaluate

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/content/semeval/src/clarity/predict.py", line 260, in <module>
    main()
  File "/content/semeval/src/clarity/predict.py", line 134, in main
    first_ckpt = torch.load(ckpt_paths[0], map_location="cpu", weights_only=False)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/serialization.py", line 1500, in load
    with _open_file_like(f, "rb") as opened_file:
         ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/serialization.py", line 768, in _open_file_like
    return _open_file(name_or_buffer, mode)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/serialization.py", line 749, in __init__
    super().__init__(open(name, mode))  # noqa: SIM115
                     ^^^^^^

## 6. Ensemble (3 seeds)
Train seeds 43 & 44, then average logits.

In [32]:
for seed_cfg in ['configs/deberta_v3_large_seed43.yaml',
                 'configs/deberta_v3_large_seed44.yaml']:
    print(f"\n{'='*60}\nTraining: {seed_cfg}\n{'='*60}")
    !python -m clarity.train --config {seed_cfg} --data data/train.csv --dev data/dev.csv --task evasion


Training: configs/deberta_v3_large_seed43.yaml
[2026-02-20 06:40:26] INFO clarity: Configuration: {'config': 'configs/deberta_v3_large_seed43.yaml', 'data': 'data/train.csv', 'dev': 'data/dev.csv', 'task': 'evasion', 'model_name': 'microsoft/deberta-v3-large', 'max_length': 384, 'batch_size': 8, 'grad_accum': 2, 'lr': 1e-05, 'weight_decay': 0.01, 'warmup_ratio': 0.06, 'epochs': 5, 'patience': 3, 'dropout': 0.1, 'seed': 43, 'output_dir': 'checkpoints/deberta_v3_large_seed43', 'use_focal_loss': False, 'focal_gamma': 2.0, 'use_class_weights': False, 'alpha': 0.7, 'consistency_beta': 0.1, 'num_workers': 0, 'fp16': False, 'precision': 'bf16', 'device': None, 'label_smoothing': 0.05, 'debug_text': False}
[2026-02-20 06:40:26] INFO clarity: Using device: cuda
[2026-02-20 06:40:26] INFO clarity: CUDA: TF32 enabled for matmul and cuDNN
[2026-02-20 06:40:26] INFO clarity: CUDA: Flash/mem-efficient/math SDP all enabled
[2026-02-20 06:40:26] INFO clarity: GPU: NVIDIA A100-SXM4-80GB (85.1 GB)
[202

In [ ]:
!python -m clarity.ensemble \
    --ckpts \
        checkpoints/deberta_v3_large/best_model.pt \
        checkpoints/deberta_v3_large_seed43/best_model.pt \
        checkpoints/deberta_v3_large_seed44/best_model.pt \
    --data data/dev.csv \
    --out submissions/ensemble_predictions.csv \
    --task evasion \
    --evaluate

[2026-02-20 06:36:43] INFO clarity: Ensemble: 3 checkpoints
[2026-02-20 06:36:43] INFO clarity:   [1] checkpoints/deberta_v3_large/best_model.pt
[2026-02-20 06:36:43] INFO clarity:   [2] checkpoints/deberta_v3_large_seed43/best_model.pt
[2026-02-20 06:36:43] INFO clarity:   [3] checkpoints/deberta_v3_large_seed44/best_model.pt
[2026-02-20 06:36:43] INFO clarity: Using device: cuda
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/content/semeval/src/clarity/ensemble.py", line 180, in <module>
    main()
  File "/content/semeval/src/clarity/ensemble.py", line 89, in main
    first_ckpt = torch.load(args.ckpts[0], map_location="cpu", weights_only=False)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/serialization.py", line 1500, in load
    with _open_file_like(f, "rb") as opened_file:
         ^^^^^^^

## 7. Download

In [ ]:
if IN_COLAB:
    from google.colab import files
    files.download('submissions/predictions.csv')
    if Path('submissions/ensemble_predictions.csv').exists():
        files.download('submissions/ensemble_predictions.csv')
else:
    print(f"Results: {os.path.abspath('submissions/')}")

FileNotFoundError: Cannot find file: submissions/predictions.csv